In [1]:
#load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from math import sqrt

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from scipy import sparse
from scipy.stats import uniform, randint
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder  
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, recall_score, precision_score, f1_score, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

import  lightgbm as lgbm 
import xgboost as xgb 
import catboost as cat 

In [3]:
#load data
df = pd.read_csv("PleaseFundThis.csv")

In [4]:
#check data
df.head()

,project_id,project_name,date_launched,duration_days,goal_$,percent_raised,amt_pledged_$,major_category,minor_category,project_update_count,...,highest_pledge_reward_$,total_count_of_pledge_levels,project_success,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,805069526,Big Bully BBQ,19/04/2012,60,12000.0,29.67,3560.0,Food,Food,0,...,"2,500",9,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1046206236,Gary Graves New Recording,19/04/2012,60,2500.0,31.80,795.0,Music,Country & Folk,0,...,500,7,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7014665777,Newgrounds: The Forsaken City,20/04/2012,60,30000.0,61.26,18378.0,Film & Video,Webseries,8,...,"5,000",17,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,208975283,Children of the Elements - the Sustainability ...,20/04/2012,60,12500.0,48.18,6022.0,Publishing,Children's Book,12,...,"2,000",13,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,307345220,Synchronized Chaos International Magazine's Ou...,20/04/2012,60,300.0,133.33,400.0,Publishing,Periodical,0,...,100,4,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#summary
df.describe()

,project_id,duration_days,goal_$,percent_raised,amt_pledged_$,project_update_count,number_of_pledgers,comments_count,facebook_friends_count,total_count_of_pledge_levels,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
count,2.844700e+04,28447.000000,2.844700e+04,28447.000000,2.844700e+04,28447.000000,28447.00000,28447.000000,17886.000000,28447.000000,0.0,7.000000,6.000000,4.000000,3.000000,3.000000,3.000000
mean,4.126527e+09,32.749886,2.057456e+04,120.701537,1.019584e+04,3.218582,133.17559,30.299012,479.219445,9.203607,NaN,2024.000000,2445.833333,1306.250000,76.666667,78.333333,80.000000
std,2.935143e+09,10.979895,2.410157e+05,1757.962943,9.136661e+04,5.228219,1124.92532,740.131945,777.858283,5.229805,NaN,3341.584255,4008.199617,2462.604905,28.431204,25.658007,26.457513
min,6.022280e+05,1.000000,1.000000e+00,0.000000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,1.000000,NaN,40.000000,45.000000,45.000000,45.000000,50.000000,50.000000
25%,1.065575e+09,30.000000,2.000000e+03,5.380000,2.900000e+02,0.000000,6.00000,0.000000,0.000000,6.000000,NaN,90.000000,85.000000,71.250000,65.000000,67.500000,70.000000
50%,4.043883e+09,30.000000,5.000000e+03,72.590000,1.710000e+03,1.000000,28.00000,0.000000,221.000000,8.000000,NaN,310.000000,275.000000,90.000000,85.000000,85.000000,90.000000
75%,7.022325e+09,35.000000,1.200000e+04,113.330000,5.675000e+03,4.000000,80.00000,3.000000,596.000000,11.000000,NaN,2375.000000,3112.500000,1325.000000,92.500000,92.500000,95.000000
max,9.099942e+09,60.000000,2.147484e+07,240716.000000,8.596475e+06,147.000000,91584.00000,59463.000000,5358.000000,31.000000,NaN,8888.000000,10000.000000,5000.000000,100.000000,100.000000,100.000000


In [6]:
#get columns
df.columns.tolist()

['project_id',
 'project_name',
 ' date_launched ',
 'duration_days',
 'goal_$',
 'percent_raised',
 'amt_pledged_$',
 'major_category',
 'minor_category',
 'project_update_count',
 'city',
 'region',
 'number_of_pledgers',
 'comments_count',
 ' avg_amt$_per_pledger ',
 'project_has_video',
 'project_has_facebook_page',
 'facebook_friends_count',
 'project_has_pledge_rewards',
 ' lowest_pledge_reward_$ ',
 ' highest_pledge_reward_$ ',
 ' total_count_of_pledge_levels ',
 'project_success',
 'Unnamed: 23',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29']

In [7]:
#drop unnamed columns
df = df.drop(['Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','Unnamed: 27','Unnamed: 28','Unnamed: 29'], axis=1)

In [8]:
#drop descriptive metrics
df = df.drop([' lowest_pledge_reward_$ ',' highest_pledge_reward_$ ',' total_count_of_pledge_levels ', 
              ' avg_amt$_per_pledger '],axis=1)

In [9]:
df.shape

(28447, 19)

In [10]:
#check for missing data
df.isnull().sum()

project_id                        0
project_name                      0
 date_launched                    0
duration_days                     0
goal_$                            0
percent_raised                    0
amt_pledged_$                     0
major_category                    0
minor_category                    0
project_update_count              0
city                              0
region                            0
number_of_pledgers                0
comments_count                    0
project_has_video                 0
project_has_facebook_page         0
facebook_friends_count        10561
project_has_pledge_rewards        0
project_success                   0
dtype: int64

In [11]:
#drop facebook friends count
df = df.drop('facebook_friends_count',axis=1)

In [11]:
#drop city, drop minor category, project id and name, percent raised
df = df.drop('facebook_friends_count',axis=1)

array([' DC', ' TX', ' CA', ' Hungary', ' GA', ' NY', ' FL', ' NJ', ' WA',
       ' IL', ' KY', ' Australia', ' CO', ' MA', ' NV', ' AZ',
       ' Netherlands', ' NM', ' Ecuador', ' MD', ' UT', ' MN', ' OR',
       ' MO', ' OK', ' TN', ' WI', ' CT', ' ID', ' IN', ' ME', ' RI',
       ' AR', ' OH', ' PA', ' NC', ' Canada', ' Spain', ' Taiwan', ' SD',
       ' Mt', ' LA', ' Germany', ' MI', ' IA', ' Burkina Faso', ' China',
       ' Finland', ' NE', ' MT', ' VA', ' India', ' Thailand', ' Iceland',
       ' Sweden', ' DE', ' VT', ' NH', ' SC', ' AL', ' AK', ' Japan',
       ' ND', ' MS', ' United Kingdom', ' Ireland', ' KS', ' HI',
       ' New Zealand', ' Russia', ' Guatemala', ' Scotland', ' Denmark',
       ' France', ' Cambodia', ' South Africa', ' Poland', ' Austria',
       ' Italy', ' Israel', ' Portugal', ' Bangladesh', ' WV',
       ' Switzerland', ' Haiti', ' Costa Rica', ' Mexico', ' Cuba',
       ' Turkey', ' Colombia', ' Serbia', ' Lebanon',
       ' Trinidad and Tobago', ' W

array(['Washington', 'Austin', 'Los Angeles', ..., 'Washougal',
       'Carrizozo', 'Holstebro'], dtype=object)

array(['Washington', ' DC', 'Austin', ..., 'Washougal', 'Carrizozo',
       'Holstebro'], dtype=object)

In [ ]:
#check for target ratio